In [ ]:
# an example of how to carry out Domain Adaptation between a source (MNIST) Dataset and target (Fashion-MNIST) Dataset.
#just switch dataset combinations to carry out DA between different pairs of datasets ( also change the number of classes accordingly)
import tensorflow as tf
import pandas as pd
from keras import models
from keras.layers.convolutional import Convolution2D
from keras.layers.convolutional import MaxPooling2D
from keras.layers.core import Activation
from keras.layers.core import Flatten
from keras.layers.core import Dense
from keras.models import Sequential
from keras.optimizers import Adam
from keras.utils import np_utils
from sklearn.model_selection import train_test_split
from sklearn.datasets import fetch_openml
from sklearn.utils import shuffle
import numpy as np

#Training a LeNet5 on the source MNIST Dataset 
mnisttt = fetch_openml('mnist_784') 

mnist = pd.concat([pd.DataFrame(mnisttt.data), pd.DataFrame({ 'target' : mnisttt.target})], axis = 1)

mnist =shuffle(mnist, random_state = 2)

source_dataset = pd.DataFrame(mnist.iloc[:,0:784]).to_numpy()

source_labels = pd.DataFrame(mnist.target).to_numpy()

# Reshape the data to a (70000, 28, 28) tensor
source_data = source_dataset.reshape((source_dataset.shape[0], 28, 28 ))
# Reshape the data to a (70000, 28, 28, 1) tensord
source_data = source_data[:, :, :, np.newaxis]

# Scale values from range of [0-255] to [0-1]
scaled_source_data = source_data / 255.0

# Split the dataset into training and test sets
(train_data, test_data, train_labels, test_labels) = train_test_split(
    scaled_source_data,
    source_labels.astype("int"), 
    test_size = 1/7)

# Tranform training labels to one-hot encoding
train_labels = np_utils.to_categorical(train_labels, 10)

# Tranform test labels to one-hot encoding
test_labels = np_utils.to_categorical(test_labels, 10)

# Create a sequential model
model = Sequential()

# Add the first convolution layer
model.add(Convolution2D(
    filters = 20,
    kernel_size = (5, 5),
    padding = "same",
    input_shape = (28, 28, 1)))

# Add a ReLU activation function
model.add(Activation(
    activation = "relu"))

# Add a pooling layer
model.add(MaxPooling2D(
    pool_size = (2, 2),
    strides =  (2, 2)))

# Add the second convolution layer
model.add(Convolution2D(
    filters = 50,
    kernel_size = (5, 5),
    padding = "same"))

# Add a ReLU activation function
model.add(Activation(
    activation = "relu"))

# Add a second pooling layer
model.add(MaxPooling2D(
    pool_size = (2, 2),
    strides = (2, 2)))

# Flatten the network
model.add(Flatten())

# Add a fully-connected hidden layer
model.add(Dense(500))

# Add a ReLU activation function
model.add(Activation(
    activation = "relu"))

# Add a fully-connected output layer
model.add(Dense(10))

# Add a softmax activation function
model.add(Activation("softmax"))

adam = Adam(lr=1e-3, decay=1e-6)

# Compile the network
model.compile(
    loss = 'categorical_crossentropy', 
    optimizer = adam,
    metrics = ['accuracy'])

# Train the model 
model.fit(
    train_data, 
    train_labels, 
    batch_size = 128, 
    nb_epoch = 20,
	  verbose = 1)

# Evaluate the model
(loss, accuracy) = model.evaluate(
    test_data, 
    test_labels,
    batch_size = 128, 
    verbose = 1)

# Print the model's accuracy
print(accuracy)

#save trained network
model.save('train_mnist.h5')


#load saved network and weights
model =models.load_model('train_mnist.h5')
model.load_weights('train_mnist.h5')
new_model = Sequential() 

for layer in model.layers:
    new_model.add(layer) # copying previously saved network

for layer in new_model.layers[:-5]:
    layer.trainable = False  #for fine-tuning on target dataset need to freeze top three layers

#Fine-tuning pre-trained network on target Fashion-MNIST Dataset
fmnistt = fetch_openml('Fashion-MNIST') 
    
fmnist = pd.concat([pd.DataFrame(fmnistt.data), pd.DataFrame({ 'target' : fmnistt.target})], axis = 1)

fmnist =shuffle(fmnist, random_state = 2)

target_dataset = pd.DataFrame(fmnist.iloc[:,0:784]).to_numpy()

target_labels = pd.DataFrame(mnist.target).to_numpy()

# Reshape the data to a (70000, 28, 28) tensor
target_data = target_dataset.reshape((target_dataset.shape[0], 28 , 28))

# Reshape the data to a (70000, 28, 28, 1) tensord
target_data = target_data[:, :, :, np.newaxis]

# Scale values from range of [0-255] to [0-1]
scaled_target_data = target_data / 255.0

# Split the dataset into training and test sets
(train_data, test_data, train_labels, test_labels) = train_test_split(
    scaled_target_data,
    target_labels.astype("int"), 
    test_size = 1/7)

# Tranform training labels to one-hot encoding
train_labels = np_utils.to_categorical(train_labels, 10)

# Tranform test labels to one-hot encoding
test_labels = np_utils.to_categorical(test_labels, 10)


adam = Adam(lr=1e-3, decay=1e-6)

# Compile the network
new_model.compile(
    loss = 'categorical_crossentropy', 
    optimizer = adam,
    metrics = ['accuracy'])

# Train the model 
new_model.fit(
    train_data, 
    train_labels, 
    batch_size = 128, 
    epochs = 10,
	  verbose = 1)

# Evaluate the model
(loss, accuracy) = new_model.evaluate(
    test_data, 
    test_labels,
    batch_size = 128, 
    verbose = 1)

# Print the model's accuracy
print('accuracy of a MNIST-pre-trained neural network on FMNIST:', accuracy)



new_model.save('train_mnist-fmnist.h5')